In [1]:
import numpy as np, pandas as pd, plotly.express as px, plotly.graph_objects as go
from scipy.signal import savgol_filter as sgf
from pathlib import Path
import scipy.optimize as opt

from SJ_Haar_CNV import base_search
from SJ_Haar_CNV import decomposition
from tests import test_data

In [2]:
X = np.concatenate([np.repeat (5,180), np.repeat (6,21), np.repeat (5,190), np.repeat (6,190), np.repeat(5,100)])+ np.random.normal(0,1,681)
l = 681
#

In [3]:
basis = base_search.generate_haar_basis(X, p0=.8, length = 21, debug = False)
basis

[[0, 0, 0, (5.309633223930893, 681), (0, 0)],
 [0, 681, 0, (0.03363223063637841, 384), (-0.04359733601012016, 297)],
 [1, 384, 0, (0.09574603534846936, 84), (-0.027128043348732988, 300)],
 [1, 296, 385, (0.0401013841544913, 200), (-0.08396227307346615, 96)],
 [2, 84, 0, (0.20215556649525007, 18), (-0.058196299445602305, 66)],
 [2, 299, 85, (0.03660005277220715, 213), (-0.09107454992153874, 86)],
 [2, 200, 385, (0.0432919240780968, 145), (-0.11492038028003879, 55)],
 [2, 95, 586, (0.1928791874526149, 20), (-0.05400617248673218, 75)],
 [3, 65, 19, (0.09304842103984708, 41), (-0.1628347368197324, 24)],
 [3, 213, 85, (0.07578766040989116, 95), (-0.06165775762160637, 118)],
 [3, 85, 299, (0.13665695491370397, 32), (-0.08508829268211755, 53)],
 [3, 145, 385, (0.0839021681507441, 71), (-0.08163454198450779, 74)],
 [3, 54, 531, (0.11009637651263607, 32), (-0.1651445647689541, 22)],
 [3, 74, 607, (0.07921180343813394, 50), (-0.16832508230603463, 24)],
 [4, 41, 19, (0.08058229640253804, 32), (-0

In [4]:
hbasis, coefficients, all_coefficients, thr, rle_cnv = decomposition.decompose (X, basis)

In [5]:
all_coefficients


array([ 1.        , -8.47577804, -2.87452694,  7.61432825, -2.5216968 ,
        2.57252875, -3.62643293, -3.00471428, -1.71802775, -2.62766765,
       -2.9958766 , -1.03737725, -1.7633117 ,  1.74278293,  1.87261286,
       -1.24805667, -1.08678192,  2.62434189, -2.11773264, -1.50590908,
        0.68110351,  1.13477806, -2.05683853, -1.86013447, -3.20074761,
       -2.13902615, -0.52679212,  1.33429084, -1.03046066, -1.76036044,
       -2.44465814, -1.52248519, -0.71722804, -1.76404323, -2.62003125,
       -3.21154622, -3.05718291, -0.87012818,  1.94061597, -1.74178384,
       -1.1744972 , -0.68692234, -1.87564807, -1.13241813,  1.34025081,
       -1.30909079, -1.6547186 , -1.18019128, -1.38449467, -1.37357582,
       -1.07124636, -1.08361554])

In [6]:
fig = go.Figure()
#fig.add_trace(go.Scatter(x=df['location'], y=df['raw'], name='Raw Signal', mode='markers',marker=markers))
fig.add_trace(go.Scatter(y = X, mode = 'markers'))
fig.add_trace (go.Scatter (y = decomposition.generate_function_from_wavelets (coefficients, basis, l)))
#ind = np.concatenate([[0], np.where(coefficients > thr)[0]])
#allind = np.arange(len(basis))
fig.add_trace (go.Scatter (y =decomposition.generate_function_from_wavelets (all_coefficients, basis, l)))

In [22]:
def rle(s):
    counts = {}
    for c in s:
        if counts.get(c) is None:
            counts[c] = 1
        else:
            counts[c] = counts[c] + 1
    print (counts)
    return [(k,counts[k]) for k in counts.keys()]

y = decomposition.generate_function_from_wavelets (coefficients, basis,l)

rle(y)

{5.078637245913166: 341, 5.641300225518808: 1, 5.301840532622439: 51, 6.047972463138724: 190, 5.032975520241535: 98}


[(5.078637245913166, 341),
 (5.641300225518808, 1),
 (5.301840532622439, 51),
 (6.047972463138724, 190),
 (5.032975520241535, 98)]

In [15]:
fig = go.Figure()
v = np.abs (coefficients)


def exp_cdf (x, a):
    return 1- np.exp(-a*x)

popt, pcov = opt.curve_fit(exp_cdf, np.sort(v), np.linspace(0, 1, len(v)), p0=[1])

thr = -np.log(1 - 1/len(coefficients))/popt[0]


fig.add_trace(go.Scatter (x = np.sort(v), y = np.linspace (0,1,len(v)), mode = 'markers'))

c = np.linspace (0, 18, 100)
fig.add_trace(go.Scatter (x = c, y = exp_cdf(c, *popt)))
fig.add_trace(go.Scatter (x = (thr+1,thr+1), y = (0.1, 0.99)))


In [14]:
popt

array([0.90384146])

In [111]:
fig = go.Figure()
fig.add_trace(go.Scatter (x = np.sort(np.log(v)), y = np.linspace (0,1,len(v)), mode = 'markers'))